---
title: Aggregate data from NetCDF to DHIS2 organisation units
short_title: Aggregata data from NetCDF
---

Load data from NetCDF using [earthkit](https://ecmwf.github.io/earthkit-website/) and aggregate the data to DHIS2 organization units.  

In [42]:
import earthkit.data
from earthkit.transforms import aggregate
from dhis2eo import dataArrayToJson 

Load a NetCDF file using earthkit. See [more examples](https://earthkit-data.readthedocs.io/en/latest/examples/) for how you can load data with eartkit. 

In [43]:
file = "data/era5-land-daily-mean-temperature-2m-july-2025-sierra-leone.nc"
data = earthkit.data.from_source("file", file)

To display the contents of the dataset we can convert it to an [xarray](https://xarray.dev). It shows that the file includes 3 dimensions (latitude, longitude and valid_time) and one data variable "t2m" (temperature at 2m above sea level). The data source is European Centre for Medium-Range Weather Forecasts ([ECMWF](https://www.ecmwf.int)). 

In [44]:
data.to_xarray()

<xarray.Dataset> Size: 209kB
Dimensions:     (valid_time: 31, latitude: 41, longitude: 41)
Coordinates:
    number      int64 8B ...
  * latitude    (latitude) float64 328B 10.0 9.9 9.8 9.7 9.6 ... 6.3 6.2 6.1 6.0
  * longitude   (longitude) float64 328B -14.0 -13.9 -13.8 ... -10.2 -10.1 -10.0
  * valid_time  (valid_time) datetime64[ns] 248B 2025-07-01 ... 2025-07-31
Data variables:
    t2m         (valid_time, latitude, longitude) float32 208kB dask.array<chunksize=(31, 41, 41), meta=np.ndarray>
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-08-07T14:32 GRIB to CDM+CF via cfgrib-0.9.1...

eartkit can also be used to load the organisation units from DHIS2 that we [saved as a GeoJSON file](organization-units). 

In [45]:
district_file = "data/sierra-leone-districts.geojson"
features = earthkit.data.from_source("file", district_file)

We can display the first feature to see the information we have for each org unit. For the aggregation, we are particularly interested in the id and the geometry (polygon) of the org unit. 

In [46]:
features[:1]

[type                                                                Polygon
 id                                                              O6uvpzGd5pu
 name                                                                     Bo
 hasCoordinatesDown                                                     True
 hasCoordinatesUp                                                      False
 level                                                                     2
 grandParentParentGraph                                                     
 grandParentId                                                              
 parentGraph                                                     ImspTQPwCqd
 parentId                                                        ImspTQPwCqd
 parentName                                                     Sierra Leone
 dimensions                                                              { }
 weight                                                                    1

To aggregate the data to the org unit features we use the aggregate package of [earthkit-transforms](https://earthkit-transforms.readthedocs.io). We keep the daily period type and only aggregate the data spatially to the org unit features. mask_dim is dimension (org unit id) that will be created after the reduction of the spatial dimensions (longitude/latitude grid). 

In [47]:
agg_data = aggregate.spatial.reduce(data, features, mask_dim="id")

The aggregated data is returned as an xarray with two dimensions (id and valid_time), and the same temperature vaiable. 

In [48]:
agg_data

<xarray.Dataset> Size: 2kB
Dimensions:     (valid_time: 31, id: 13)
Coordinates:
    number      int64 8B 0
  * valid_time  (valid_time) datetime64[ns] 248B 2025-07-01 ... 2025-07-31
  * id          (id) object 104B 'O6uvpzGd5pu' 'fdc6uOvgoji' ... 'at6UHUQatSo'
Data variables:
    t2m         (id, valid_time) float32 2kB 296.8 297.3 297.3 ... 298.6 298.1
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-08-07T14:32 GRIB to CDM+CF via cfgrib-0.9.1...

Next, we select the variable we would like to import to DHIS2 (t2m). We also convert temperatures in kelvin to celcius by subtracting 273.15 from the values.

In [59]:
dataArray = agg_data['t2m'] - 273.15

This returns a multidimensional xarray.dataArray with the two dimensions (id and valid_time). We rename the dimensions so we have one named "orgUnit" and one named "period".

In [60]:
formatted = dataArray.rename(id='orgUnit', valid_time='period')

The two dimensions are "stacked" into one dimension using the [xarray stack method](https://docs.xarray.dev/en/latest/generated/xarray.DataArray.stack.html): 

In [61]:
stacked = formatted.stack(index=[...])

This can be translated into a JSON string with the dataArrayToJson function.

In [57]:
json = dataArrayToJson(stacked);

We can display the first part of this JSON string to see that we have one temperatue value for each org unit and period combination.

In [64]:
json[:500]

'[{"orgUnit": "O6uvpzGd5pu", "period": "2025-07-01", "value": 23.6839599609375}, {"orgUnit": "fdc6uOvgoji", "period": "2025-07-01", "value": 23.95904541015625}, {"orgUnit": "lc3eMKXaEfw", "period": "2025-07-01", "value": 24.52099609375}, {"orgUnit": "jUb8gELQApl", "period": "2025-07-01", "value": 23.06378173828125}, {"orgUnit": "PMa2VCrupOd", "period": "2025-07-01", "value": 24.454498291015625}, {"orgUnit": "kJq2mPyFEHo", "period": "2025-07-01", "value": 23.270477294921875}, {"orgUnit": "qhqAxPST'